# Probability for Rendering

> This lab intends to familiar you with basic math components in rendering, which should have been covered in *Probability* and *Mathematical Analysis*.

Hi! Welcome to Computer Graphics at ShanghaiTech University!
In this lab, we'll guide you through some simple yet important factors in building a *Physically-based Render*.
It might also serve as an excuse for you to review probability theory.

Please execute the following code section to make sure that all dependencies are installed.

In [ ]:
import numpy as np
import pandas as pd
import statistics
import math
import random

import scipy.integrate as integrate
import scipy.special as special
import scipy.optimize as optimize
from scipy.stats import norm

np.random.seed(42)

## Estimator Basics

You might have heard of the words *biased*, *consistent* or *unbiased* along the way of learning probability.
Generally, these words are used to describe *estimators*.
**Estimators are random variables**. The reason they are called *estimators* is that, they are used to estimate some *underlying values*.
So for arbitrary *estimator*, we can take, for example, expectation and variance on them.
Don't worry if you don't understand what do me mean by *underlying values*, by the end of this lab, you'll get a comprehensive understanding of estimator.

Here is a simple and incomplete abstraction of normal distribution $\mathcal{N}(\mu_i, \sigma_i^2)$.

In [ ]:
class Normal:
    def __init__(self, n: int = 100) -> None:
        self.n_ = n  # num samples
        self.mu_ = math.sqrt(42)
        self.sigma_ = 24
        self.samples_ = np.random.normal(self.mu_, self.sigma_, self.n_)

    def sample(self) -> float:
        self.n_ -= 1
        result = self.samples_[self.n_]
        return result

Suppose that we're in a new universe with unknown light speed $c$, each sample (experiment) of the previous class will give us a *measure* of light speed.
You, our explorer, have limited resource to perform experiments, that you can only perform experiments for $100$ times.

But you are asked by your commander to give the prediction of $c^2$, which is unexpectingly important in scientific computing.
Your rough plan is to design an estimator $Y$ that can give you the prediction, i.e., $\mathbb{E}\left[Y\right] = c^2$.
Your commander told you that, *all estimator design begins with writing this form of formula*.

After some rough thinking, you think there are three approaches to design the estimator.
And you acknowledged that calling `sample` for $n$ times is equivalent to producing $n$ i.i.d. random variables $X_i$ where $\mathbb{E}\left[X_i\right] = c$.
For now, you don't have to understand what *biased*, etc. really means.
$$
\left\{
\begin{aligned}
  Y_1 &= \dfrac{1}{100}\sum_{i = 1}^{100}{X_i^2} &\text{(biased)} \\
  Y_2 &= \left(\dfrac{1}{100}\sum_{i = 1}^{100}{X_i}\right)^2 &\text{(consistent)} \\
  Y_3 &= \left(\dfrac{1}{50}\sum_{i = 1}^{50}{X_i}\right) \times \left(\dfrac{1}{50}\sum_{i = 51}^{100}{X_i}\right) &\text{(unbiased)} \\
\end{aligned}
\right.
$$

As an appetizer, your task is to fill the following Python functions with the formulas, which will give us the estimation of $c^2$.

In [ ]:
def biased(normal: Normal) -> float:
    result = []
    # TODO: your implementation of the *biased* estimator above
    for _ in range(100):
        pass
    return statistics.mean(result)


def consistent(normal: Normal) -> float:
    result = []
    # TODO: your implementation of the *consistent* estimator above
    for _ in range(100):
        pass
    return math.pow(statistics.mean(result), 2)


def unbiased(normal: Normal) -> float:
    # TODO: your implementation of the *unbiased* estimator above
    # First estimator
    result = []
    for _ in range(50):
        pass
    I1 = statistics.mean(result)

    # Second estimator
    result = []
    for _ in range(50):
        pass
    I2 = statistics.mean(result)

    return I1 * I2


Great! As a numerical verification, execute the following code,

In [ ]:
# A sample-based mean/variance estimator
def test(func, n: int = 1000) -> float:
    result = []
    for _ in range(n):
        result.append(func())
    return (statistics.mean(result), statistics.variance(result))


pd.options.display.float_format = '{:.2f}'.format
df = pd.DataFrame(index=['biased', 'consistent',
                  'unbiased'], columns=['mean', 'variance'])
df.loc['biased'] = test(lambda: biased(normal=Normal()))
df.loc['consistent'] = test(lambda: consistent(normal=Normal()))
df.loc['unbiased'] = test(lambda: unbiased(normal=Normal()))
print(df)


The performance discrepancy is obvious since the ground truth of $c^2$ is $42$, only the *unbiased* estimator approaches the result.

But why is that?

Sure, let's take their expectation respectively to observe the result. Recall that,
$$
  \dfrac{1}{n}\sum_{i = 1}^{n} \mathcal{N}(\mu_i, \sigma_i^2) = \mathcal{N}\left(\dfrac{1}{n}\sum_{i = 1}^{n}{\mu_i}, \dfrac{1}{n^2}\sum_{i = 1}^{n}{\sigma_i^2}\right)
$$
and
$$
  \mathbb{V}\left[X\right] = \mathbb{E}\left[X^2\right] - \mathbb{E}^2\left[X\right]
$$

Then let's test our estimators,
$$
\begin{aligned}
  \mathbb{E}\left[Y_1\right] &= \dfrac{1}{100}\sum_{i = 1}^{100}{\mathbb{E}\left[X_i^2\right]} \\
                             &= \dfrac{1}{100}\sum_{i = 1}^{100}{\left(\mathbb{V}\left[X_i\right] + \mathbb{E}^2\left[X_i\right]\right)} \\
                             &=  c^2 + \dfrac{1}{100}\sum_{i = 1}^{100}{\mathbb{V}\left[X_i\right]} \approx 576 + 41 = 617
\end{aligned}
$$

At this point, I believe you have understood what *biased estimator* means, which is, $\mathbb{E}\left[{Y_i}\right] - c^2 > 0$, and no matter how many more samples we obtain, the bias will be pertained, which is the variance of all samples.

We could introduce the term *consistent estimator* now. **Asymptotically**, when the number of samples we can obtain converges to $+\infty$, $\mathbb{E}\left[Y_i\right] = c^2$. Here is the example,
Let $X = \sum_{i} X_i = \mathcal{N}\left(\overline{\mu}, \overline{\sigma^2}/n\right)$,
$$
  \mathbb{E}\left[Y_2\right] = \mathbb{E}\left[X^2\right] = \mathbb{V}\left[X\right] + \mathbb{E}^2\left[X\right] = c^2 + \dfrac{\sigma^2}{n}
$$
Then when $n \rightarrow +\infty$, the biased is eliminated, which corresponds to the definition of *consistent estimator*.
However, whatever the parameter `n` of the `test` function is (not the `n` in `class Normal`, there is a crucial difference between these two $n$, why?), we will not obtain the correct result, since for a specific `n`, the estimator is biased at all.

Consistent estimator is great! But we deserve somehow more. For the unbiased estimator, $\forall n \ge 1, \mathbb{E}\left[Y_3\right] = c^2$. The proof is yet easy,
$$
  \mathbb{E}\left[Y_3\right] = \mathbb{E}\left[I_1\right] \times \mathbb{E}\left[I_2\right] = c^2.
$$
where $I_1$ and $I_2$ are the independent r.v.s generated from disjoint ranges of $X_i$.

Before the end of this section, there is one thing that worth mentioning. *Biased* do not necessarily imply that the estimator is inefficient.
Although the previous example's bias is due to inadvertent wrong design.
In real cases, sometimes bias are introduced intentionally to obtain low variance,
sometimes bias are difficult to mitigate, or it takes a lot of effort to eliminate, so people just leave it as it is.
Anyway, you'll meet a lot of biased but efficient estimators along the way, don't panic.

### Exercise: Unbiased Estimator for Expoential

Here's a simple conclusion. Strategy makes a difference. At this point, you should at least understand what are estimators, what are *biased*, *consistent* and *unbiased* estimators.
As for designing estimators, which is an extremely difficult and interesting field (not just) in Computer Graphics, I'll present a more interesting example here.

Your commander is very satisfied with your estimation, while traveling through a patch of interstellar dust, he asked you to give another *unbiased* estimation to $\exp(c)$, i.e., $e^c$.
Unlike the last time, this time you have more resources to conduct more experiments.

In introducing the techniques to solve this problem, we need to retrospect some basics.

Suppose we have an estimator (r.v.) $X$. Recall that $X$ is a (measurable) map. Yes, random variables are not variables, they are maps.
And there can have a *distribution* defined on its *range*. Since $X$ is a map, and maps can be composed, i.e., $f \circ g$.
Any *valid* functions applied on $X$ can form a new random variable.
For example, $f(X)$ is yet another random variable.
Further, $X \cdot Y$ is a new r.v., $p(X)$ is yet r.v., where $p$ is its distribution function.

Then, our estimator design for $\exp(c)$ is $X$, where $\mathbb{E}\left[X\right] = \exp(c)$. Then, perform Taylor series expansion on $\exp(c)$.
$$
  \exp(c) = 1 + \dfrac{c}{1!} + \dfrac{c^2}{2!} + \dfrac{c^3}{3!} + \cdots \; \text{(countable series)}
$$
$X$ can be decomposed like this, $\mathbb{E}\left[X\right] = 1 + \sum_{i = 1}^{\infty}{\mathbb{E}\left[X_i\right]}$.
So our task, actually, is to evaluate this infinite series. But we don't have infinitely many resources to use! Here is where the trick comes.

We let $\mathbb{E}\left[X_i\right] = \mathbb{E}\left[I_i \cdot X_i'\right]$,
where $I_i$ is an indicator variable controlling whether this term $X_i$ should be taken into the summation.
Since $\mathbb{E}\left[I_i \cdot X_i'\right] = 1 \cdot X_i' \cdot P(I_i = 1)$, $X_i' = X_i P(I_i)^{-1}$.
**For any** series of indicator variables $\{I_i\}_{i = 1}^{+\infty}$, this infinite series will evaluate to a correct result.

We somehow expect that the series' evaluation process can be terminated in some $i$, then let $K \sim \mathrm{Geo}(p)$, and $I_i = [K > i]$, written in [Iverson bracket](https://www.wikiwand.com/en/Iverson%20bracket), we obtain the unbiased estimator of $\exp(c)$, which is
$$
\begin{aligned}
  \mathbb{E}\left[X\right] &= \exp(c) \\  
    &= 1 + \sum_{i = 1}^{+\infty}{I_i \dfrac{X_i}{P(I_i = 1)}}
    = 1 + \sum_{i = 1}^{K-1}{\dfrac{X_i}{P(K > i)}}
\end{aligned}
$$
where $\mathbb{E}\left[X_i\right] = \dfrac{c^i}{i!}$, whose unbiased estimator design is known from our previous exercise, and $P(K > i) = (1 - p)^i, \forall i \ge 1$.

Further, one way to obtain (sample) $K$ is to actually simulate the process, where *success* means *termination*. Try to implement it here!

In [ ]:
class Normal:
    def __init__(self, mu: float) -> None:
        self.mu_ = mu
        self.sigma_ = 1.0

    def sample(self) -> float:
        return np.random.normal(self.mu_, self.sigma_)


def rr(func) -> float:
    p = 0.1       # terminate with a prob 0.1
    result = 1.0  # 1 + ... (in Taylor series)
    i = 0
    while True:
        i += 1
        if random.random() <= p:
            break
        result += (func(i) / math.pow((1 - p), i))
    return result


def exp_series(func, i: int) -> float:
    # TODO: fill in your implementation here
    result = 1.0
    for j in range(i):
        pass
    return result


mu = 2.0
print(f'the estimation of [exp({mu})={math.exp(mu):.4f}] is', test(
    lambda: rr(lambda i: exp_series(func=Normal(mu).sample, i=i)))[0])

This approach is called [Russian Roulette](https://www.wikiwand.com/en/Russian_roulette), abbreviate to RR.
RR is a such powerful tool to construct estimator on infinite series that you'll see its application later in your rendering homework.

In conclusion, we've constructed an unbiased estimator to estimate the form like $\exp(c)$ with RR.
Note that we already have an unbiased estimator for $\forall i \ge 1, c^i$.
$c$ is not necessarily a constant. We'll see an example later.

## Monte Carlo Integration

Monte Carlo Integration is a realization of *estimator design* and *numerical integration*, which designs estimator on a broad range of integral.
To understand how powerful the Monte Carlo Integration is, we start with the definition of *numerical integration*.

Unlike our homework helper (*Mathematica*) in the mathematical analysis course, which is symbolic integration, *numerical integration* intends to "calculate the numerical value of a definite integral".
In 1-D case, we might want,
$$
  c = \int_{a}^{b}{f(x) \, \mathrm{d} x},
$$
which can be abstracted to $I_{\mathrm{numerical}}\left[\mathbb{R}, [a, b] \subset \mathbb{R}, f\right] \rightarrow \mathbb{R}$, where $f$ might at least defined on $[a, b]$.
Note that this function $I_{\mathrm{numerical}}$ accepts several things, a *set*, the *integral domain* and a *real-valued function* on this domain.

A common implementation of $I_{\mathrm{numerical}}$ is to partition $[a, b]$ into disjoint *subintervals*, then perform some kinds of summations.
We'll neglect the detail here because Monte Carlo Integration is somehow completely different.
Those who are interested can refer to the explanation of numerical integration on [Wikipedia](https://www.wikiwand.com/en/Numerical_integration).

To further introduce Monte Carlo Integration to you, you need to understand why we need it.
It is what motivation that drives us away from these simple and intuitive numerical integration approaches?
You will be amazed by its *power*, *elegance*, and *simplicity*.

Again, Monte Carlo Integration is the realization of *estimator design*. You should ask two questions,
- What is the underlying value?
- How can we design the estimator?

The underlying value, to simplify, is a definite integral where $c = \int_{a}^{b}{f(x) \, \mathrm{d}x}$ (actually, to apply Monte Carlo Integration to Rendering, readers might need to understand integration on general measure space).
Recall that we expect an estimator $Y$ such that,
$$
\begin{aligned}
  \mathbb{E}\left[Y\right] &= c = \int_{a}^{b}{f(x) \, \mathrm{d}x} \\
    &= \int_{a'}^{b'}{ y \cdot p(y) \, \mathrm{d}y} \\
\end{aligned}
$$
to align these two integrals, $a' = a, b' = b$, most importantly,
$$
  Y = \dfrac{f(X)}{p(X)} = g(X).
$$
where $X$ can be **any** valid r.v. To verify,
$$
  \mathbb{E}\left[g(X)\right] = \int_{a'}^{b'}\dfrac{f(x)}{\cancel{p(x)}} \cdot \cancel{p(x)} \, \mathrm{d}x.
$$
Isn't it too simple? But it's this simple method that builds the field *Rendering*.

Let's demonstrate this process with code.

### Exercise: Monte Carlo Estimator

You and your fleet are moving through a known stretch of interstellar dust, again!
You can your commander realized that you need to perform integration on the density of dust along the way you are traveling to estimate and reduce energy consumption, which is an integral like this,
$$
  c = \int_{0}^{4.5}{\sigma(t) \, \mathrm{d}t},
$$

Random variables from now on are abstracted into a class, which is supposed to be easy to understand.
We've provided three examples here, note that `Normal` distribution should not be used as the preceding random variable of Monte Carlo estimator,
since it is not bounded, otherwise the estimator is biased.

Estimators are generated by factory functions like `make_monte_carlo_estimator`.

In [ ]:
# https://docs.scipy.org/doc/scipy/tutorial/integrate.html
# Use this simple but non-trivial function
def func(x): return special.jv(2.5, x)


class RandomVariable:
    """
    Definition of RandomVariable
        Not a formal definition, but straight-forward
    """

    def __init__(self): pass
    def sample(self) -> float: assert False
    def pdf(self, x: float) -> float: assert False

    def expect(self, n: int = 1000) -> float:
        # LLN
        result = 0.0
        for _ in range(n):
            result += self.sample()
        return result / n


class Uniform(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.inv_length_ = 1 / (b - a)

    def sample(self) -> float:
        return np.random.uniform(low=self.a_, high=self.b_)

    def pdf(self, x: float) -> float:
        return self.inv_length_


class Normal(RandomVariable):
    def __init__(self, mu, sigma):
        self.mu_ = mu
        self.sigma_ = sigma
        self.pdf_ = lambda x: norm.pdf(x, self.mu_, self.sigma_)

    def sample(self) -> float:
        return np.random.normal(self.mu_, self.sigma_)

    def pdf(self, x: float) -> float:
        return self.pdf_(x)


class TruncNormal(RandomVariable):
    def __init__(self, a: float, b: float):
        self.a_ = a
        self.b_ = b
        self.mu_ = (a + b) / 2
        self.sigma_ = (b - a)
        self.cdf_ = lambda x: norm.cdf(x, self.mu_, self.sigma_)
        self.pdf_ = lambda x: norm.pdf(x, self.mu_, self.sigma_)
        self.factor_ = self.cdf_(b) - self.cdf_(a)

    def sample(self) -> float:
        # rejection sampling
        sample = np.random.normal(self.mu_, self.sigma_)
        while not (self.a_ <= sample <= self.b_):
            sample = np.random.normal(self.mu_, self.sigma_)
        return sample

    def pdf(self, x: float) -> float:
        return self.pdf_(x) / self.factor_


class NumericalIntegral:
    """
    An integral can be abstracted to
        1. A function defined on Omega
        2. A (measurable) set (indicated by low, high)
    """

    def __init__(self, func, low, high) -> None:
        self.func_ = func
        self.low_ = low
        self.high_ = high
        self.max_ = None
        self.min_ = None

    def get_value(self):
        # Use general numerical integration method
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        return integrate.quad(self.func_, self.low_, self.high_)[0]

    def get_max(self):
        # Reserved for MCMC methods
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        # https://stackoverflow.com/questions/10146924/finding-the-maximum-of-a-function
        if self.max_ is None:
            self.max_ = -optimize.minimize_scalar(lambda x: -self.func_(
                x), bounds=[self.low_, self.high_], method='bounded').fun
        return self.max_

    def get_min(self):
        # Reserved for residual-tracking
        assert isinstance(self.low_, float) and isinstance(self.high_, float)
        # https://stackoverflow.com/questions/10146924/finding-the-maximum-of-a-function
        if self.min_ is None:
            self.min_ = optimize.minimize_scalar(
                self.func_, bounds=[self.low_, self.high_], method='bounded').fun
        return self.min_

    def make_monte_carlo_estimator(self, rv_factory) -> RandomVariable:
        class Estimator(RandomVariable):
            def __init__(self_):
                self_.base_rv_ = rv_factory(self.low_, self.high_)

            def sample(self_) -> float:
                # TODO: fill in your implementation here
                # by using .sample() and .pdf() and self.func_()
                pass
                # return ...
        return Estimator()


integral = NumericalIntegral(func=func, low=0.0, high=4.5)
est_uniform = integral.make_monte_carlo_estimator(Uniform)
est_tnormal = integral.make_monte_carlo_estimator(TruncNormal)
print(f'groundtruth: {integral.get_value():.4f}')
print(f'uniform:     {est_uniform.expect():.4f}')
print(f'tnormal:     {est_tnormal.expect():.4f}')


Hey! But things are not finished yet. Monte Carlo *Estimators* are simple, this is what we know. AND, one more thing,

**Monte Carlo Integration can be performed on arbitrarily high dimension.**

Repeat it for three times, Monte Carlo Integration can be performed on arbitrarily high dimension,
Monte Carlo Integration can be performed on arbitrarily high dimension,
Monte Carlo Integration can be performed on arbitrarily high dimension.

Reflected into the code, this says, the set in $\mathbb{R}$ indicated by `low` and `high` can be switched into some unknown representation.

To avoid spoilers on one of the most the interesting part of Computer Graphics, we'll not delve into this topic.
Later, we'll have the chance to utilize this awesome method to imitate the real-world magic in our personal computers!

## Exercise: Unbiased Variance Estimator

To test your understanding of the above,
we designed a simple exercise involving both Monte Carlo Integration and unbiased estimator design.

Here it is! Design an **unbiased** variance estimator and implement it!
To be specific, we asked you to design an estimator $\theta$ for a random variable $X$,
so that $\mathbb{E}\left[\theta\right] = \mathrm{Var}(X)$.

You are given $100$ i.i.d. samples of $X$, $\left\{X_i\right\}_{i=1}^{100}$.

Hint:
$$
\begin{aligned}
  \mathrm{Var}(X)
  & = \mathbb{E}\left[X^2\right] - \mathbb{E}^2\left[X\right] \\
  & = \int_{-\infty}^{\infty}{x^2 p(x) \, \mathrm{d} x} - \mathbb{E}^2\left[X\right].
\end{aligned}
$$

Answer:
Design $\theta$ so that (assume $n$ is even)
$$
  \theta = \left[\dfrac{1}{n} \sum_{i=1}^{n}X_i^2\right] -
  \left[
    \left(\dfrac{2}{n}\sum_{i=1,3,\cdots}^{n}X_i\right)
    \left(\dfrac{2}{n}\sum_{i=2,4,\cdots}^{n}X_i\right)
  \right]
$$

In [ ]:
def make_variance_estimator(rv: RandomVariable, n: int = 100) -> RandomVariable:
    class Estimator(RandomVariable):
        def __init__(self, n) -> None:
            super().__init__()
            self.n_ = n

        def sample(self) -> float:
            X = [rv.sample() for _ in range(self.n_)]
            X = np.array(X)
            return 1/self.n_*np.sum(X*X) - \
                (2/self.n_*np.sum(X[:(self.n_//2)])) * \
                (2/self.n_*np.sum(X[(self.n_//2):]))
    return Estimator(n)


print('normal calculated:', make_variance_estimator(Normal(42, 4)).expect())
print('       expected:', 16)
print('uniform calculated:', make_variance_estimator(Uniform(0, 1)).expect())
print('        expected', 1/12.0)


## Non-linear Estimation using Unbiased-Estimators

This section serves as an appendix, to combine the previously mentioned methods together, to estimate this!
$$
  \mathrm{Tr}(p \rightarrow p') = \exp\left(-\int_{0}^{t}{\sigma(t) \, \mathrm{d} t}\right)
$$
Which is the transmittance between two points. We might expect an *unbiased* estimator on this.
This formula is common in *Volume Rendering*, designing an efficient unbiased estimator of which is generally regarded as a hard task.

But, we already have the necessary tools to accomplish this task, just combine the previous functions!

In [ ]:
def make_taylor_transmittance_estimator(integral: NumericalIntegral):
    estimator = integral.make_monte_carlo_estimator(Uniform)

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            return rr(lambda i: exp_series(func=estimator.sample, i=i))
    return Estimator()


m_integral = NumericalIntegral(func=lambda x: -func(x), low=0.0, high=4.5)
tr_est_uniform = make_taylor_transmittance_estimator(m_integral)
print(f'groundtruth: {np.exp(m_integral.get_value()):.4f}')
print(f'mean:        {tr_est_uniform.expect():.4f}')
print(f'variance:    {make_variance_estimator(tr_est_uniform, 10).expect():.4f}')


## MCMC Methods

This part is completely optional, but we recommend you to execute it and check its result towards the previous method.
It is based on [this paper](https://cs.dartmouth.edu/wjarosz/publications/novak14residual.pdf),
you can refer to it for pseudo-code for further study.

Just execute this code! Experience the power of MCMC methods. You don't need to understand the algorithm right now,
but we strongly recommend that you read the relevant papers afterwards. The idea, intuition and derivation of this series of methods is ingenious.

In [ ]:
def delta_tracking(integral: NumericalIntegral, max_f: float):
    x = integral.low_
    while True:
        eta = np.random.uniform()
        x -= np.log(1 - eta) / max_f
        if x >= integral.high_:
            break
        epsilon = np.random.uniform()
        if epsilon <= integral.func_(x) / max_f:
            break
    return x


def make_delta_tracking_transmittance_estimator(integral: NumericalIntegral) -> RandomVariable:
    max_f = integral.get_max()

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            return 1 if delta_tracking(integral, max_f) > integral.high_ else 0
    return Estimator()


def make_ratio_tracking_transmittance_estimator(integral: NumericalIntegral) -> RandomVariable:
    max_f = integral.get_max()

    class Estimator(RandomVariable):
        def __init__(self) -> None:
            super().__init__()

        def sample(self) -> float:
            x = integral.low_
            T = 1
            while True:
                eta = np.random.uniform()
                x -= np.log(1 - eta) / max_f
                if x >= integral.high_:
                    break
                T *= (1 - integral.func_(x) / max_f)
            return T
    return Estimator()


def make_residual_tracking_transmittance_estimator(integral: NumericalIntegral, f_c: float) -> RandomVariable:
    class Estimator(RandomVariable):
        def __init__(self, f_c) -> None:
            super().__init__()
            self.f_c_ = f_c
            self.f_r_ = -optimize.minimize_scalar(
                lambda x: -np.abs(integral.func_(x) - f_c), bounds=[integral.low_, integral.high_], method='bounded').fun

        def sample(self) -> float:
            x = integral.low_
            Tc = np.exp(-self.f_c_ * (integral.high_ - integral.low_))
            Tr = 1.0
            while True:
                eta = np.random.uniform()
                x -= np.log(1 - eta) / self.f_r_
                if x >= integral.high_:
                    break
                Tr *= 1 - (integral.func_(x) - self.f_c_) / self.f_r_
            return Tc * Tr
    return Estimator(f_c)


tr_est_dt = make_delta_tracking_transmittance_estimator(integral)
tr_est_rt = make_ratio_tracking_transmittance_estimator(integral)
tr_est_rst_mean = make_residual_tracking_transmittance_estimator(
    integral, integral.get_value() / (integral.high_ - integral.low_))
print(f'groundtruth:  {np.exp(m_integral.get_value()):.4f}')
print(f'our mean:     {tr_est_uniform.expect():.4f}')
print(f'dt  mean:     {tr_est_dt.expect():.4f}')
print(f'rt  mean:     {tr_est_rt.expect():.4f}')
print(f'rst mean:     {tr_est_rst_mean.expect():.4f}')
print(
    f'our variance: {make_variance_estimator(tr_est_uniform, 10).expect():.4f}')
print(f'dt  variance: {make_variance_estimator(tr_est_dt, 10).expect():.4f}')
print(f'rt  variance: {make_variance_estimator(tr_est_rt, 10).expect():.4f}')
print(
    f'rst variance: {make_variance_estimator(tr_est_rst_mean, 10).expect():.4f}')


Their implementations seem easy. Well, yes!
But the derivation is rather complex, and requires you to manage the previous concepts (basic college math is sufficient for you to read this part).
You can refer to [this material](https://cs.dartmouth.edu/wjarosz/publications/novak14residual-supplemental1.pdf) for more information.
It further includes the proof of residual tracking, which is, again, the application of [control variates](https://www.wikiwand.com/en/Control_variates).
We'll only prove *Ratio-Tracking* here for you to entertain ;)

Happy grinding, and welcome to Computer Graphics!

### Proof of Ratio-Tracking

To recap, we are creating unbiased estimator for (simplified to $[0, d]$)
$$
T_r = \exp\left(-\int_{0}^{d} \mu(x) \, \mathrm{d}x\right)
$$
That is to say, we are designing an unbiased estimator $T$, such that $\mathbb{E}\left[T\right] = T_r$. $S_i$ are some i.i.d. random variables satisfying $S_i \sim \mathrm{Exp}(\overline{\mu})$, and $C_i = \sum_{j=1}^{i} S_j$, yet some random variables. Let $K$ be the maximum $i$ for $C_i \le d$. This is a random process.

Since $\mathbb{E}\left[T\right] = \mathbb{E}_{k \sim K}\left[\mathbb{E}\left[T \mid k\right]\right]$, suppose $T_k = T \mid k$, then
$$
\mathbb{E}\left[T\right] = \sum_{k=0}^{\infty}\mathbb{E}\left[T_k\right]P(K=k).
$$
We derive some terms with Mathematica, let $R_k = \mathbb{E}\left[T_k\right] P(K = k)$. Then
$$
\begin{aligned}
	R_k
	& = \int_{\Omega_k} T_k P(\cdot \mid K = k) P(K = k) \\
	& = \int_{\Omega_k} T_k P(S_i \text{ satisfying some conds}),
\end{aligned}
$$
where $\Omega_k \in \mathbb{R}^k$.

We find that $R_0 = \exp{\left(-d \mu\right)} \cdot 1$, $R = d \exp{\left(-d \mu\right)} \mu \cdot $, and... (don't panic! this is not difficult. We are just destructing $P\left(\sum_{i=1}^{k}{S_i} \le d, \sum_{i=1}^{k+1}{S_i} > d\right) \prod_{i=1}^{k}{\iota\left(\sum_{j=1}^{i}{S_j}\right)}$.
$$
\begin{aligned}
	R_k
	& = \int_{0}^{d} p_S(x_1) \iota(x_1) \int_{0}^{d - x_1} p_S(x_2) \iota(x_1 + x_2) \cdots \\
	& \cdots \int_{0}^{d - \sum_{j=1}^{k-1}{x_j}} p_S(x_k) \iota\left(\sum_{j=1}^{k}x_j\right) \\
	& \times \int_{d - \sum_{j=1}^{k-1}{x_j}}^{\infty} p_S(x_{k+1}) \, \mathrm{d} \Omega_k \\
	& \xrightarrow{\text{some magic}} \overline{\mu}^k \exp{\left(- \mu d\right)} \cdot \int_{0}^{d} \iota(z_1) \int_{0}^{d - z_1} \iota(z_2) \cdots ,
\end{aligned}
$$
where $z_i$ are $\sum_{j=1}^{i} x_j$. The *magic* is easily obtained by just multiplying the PDFs together.

Here's a non-trivial transformation that worth mentioning, how can we deal with (note that the equation 13 in the original supplementary material is incorrect, [this](https://math.stackexchange.com/questions/3778215/multiple-integral-over-a-k-dimensional-simplex) is correct ;)
$$
\int_{0}^{d} \int_{z_1}^{d} \cdots \prod_{j=1}^{k}{\iota(z_j)}
$$
Notice that $\iota(\mathbf{z})$ is symmetry over all dimensions, and we're necessarily integrating some $k$-dimensional [simplex](https://www.wikiwand.com/en/Simplex) (you understand it when you draw it). It can be transformed into
$$
R_k = \overline{\mu}^k \exp{\left(- \overline{\mu} d\right)} \left(\int_{0}^{d}{\iota(x) \, \mathrm{d} x}\right)^k {\Large{/}} k!.
$$
This equation can be easily understood in $k = 2$ using symmetry.

Then
$$
\begin{aligned}
	\mathbb{E}\left[T\right]
	& = \sum_{k=0}^{\infty}R_k = \exp(-\overline{\mu} d) \cdot \exp{\left(\overline{\mu} \int_{0}^{d} 1 - \dfrac{\mu(x)}{\overline{\mu}} \, \mathrm{d} x\right)} \\
	& = \exp{\left(- \int_{0}^{d} \mu(x) \, \mathrm{d} x\right)}
\end{aligned}
$$
Anyway, the core idea is to construct a form like,
$$
\sum_{k} \left(\int_{0}^{d} -\mu(x) \, \mathrm{d} x\right)^k {\Large/} k!
$$
though the approach is quite tricky.

## Epilogue

We have to admit, the topic of *rendering* is far from over, and the math is already becoming harder.
But one thing to mention here, so far, the math is *concrete*. We are performing integrals in trivial sets or ranges.
Once you figure out the concept through those (elaborately designed) interfaces, we can proceed to the next level,
to grind through those abstractly defined equations and integrals!
This is where the difficulty for neophyte in rendering comes from.

Please notice, we are entering a brand new mathematically defined world after this lab, not the one that you are familiar with.
We encourage you to ask yourself questions, for example, random variables are defined on *sample space*, so how do we design an estimator on this integral;
what is **dummy variable** in integral; what do we really mean by integrating on solid angle?
To build a comprehensive and mathematically correct path tracer, you should be able to ask and answer these questions.
If you encounter any difficulties, reflect your model of abstraction, and we are happy to discuss these things with you!

There are some good resources that worth reading. One we'd like to recommend here, aside from Physically Based Rendering, is
ROBUST MONTE CARLO METHODS FOR LIGHT TRANSPORT SIMULATION. Check this [meme](https://twitter.com/yiningkarlli/status/1545186689894584320) :)